In [1]:
import pickle
import re
from tqdm import tqdm
import binascii
import numpy as np
import random
import pandas as pd

In [2]:
with open("data/all_news.pkl", 'rb') as f:
    news = pickle.load(f)

In [3]:
def get_shingles(words, n=2):
    for i in range (0,len(words)-n+1):
        yield ' '.join(words[i:i+n])

In [4]:
def clean(source):
    stop_symbols = '.,!?:;-\n\r()—'

    stop_words = ('это', 'как', 'так',
                  'и', 'в', 'над',
                  'к', 'до', 'не',
                  'на', 'но', 'за',
                  'то', 'с', 'ли',
                  'а', 'во', 'от',
                  'со', 'для', 'о',
                  'же', 'ну', 'вы',
                  'бы', 'что', 'кто',
                  'он', 'она')

    return ( [x for x in [y.strip(stop_symbols) for y in source.lower().split()] if x and (x not in stop_words)] )

In [5]:
shingle_sets = {}
all_shingles = set()
i = 0
for document in tqdm(news):
    shingles_in_doc = set()
    clean_doc = clean(document['body'])
    shingles = {i for i in get_shingles(clean_doc, 5)}
    for shingle in shingles:
        crc = binascii.crc32(shingle.encode('utf-8')) & 0xffffffff
        shingles_in_doc.add(crc)
        all_shingles.add(crc)
    
    shingle_sets[i] = shingles_in_doc
    i += 1

100%|██████████████████████████████████| 20016/20016 [00:14<00:00, 1417.88it/s]


In [6]:
all_shingles = list(all_shingles)

In [7]:
docs_num = len(news)

In [9]:
sim_matrix = np.zeros((docs_num, docs_num))

In [ ]:
for i in tqdm(range(docs_num)):
    doc_1 = shingle_sets[i]
    for j in range(i+1, docs_num):
        doc_2 = shingle_sets[j]
        sim_matrix[i][j] = len(doc_1.intersection(doc_2)) / len(doc_1.union(doc_2))

## MinHashing

In [8]:
num_hashes = 15
sim_matrix = np.zeros((docs_num, docs_num))

In [9]:
def build_new_hashes():
    new_list = []
    for i in range(num_hashes):
        r = random.sample(all_shingles, len(all_shingles))
        new_list.append(r)
        
    return new_list

In [10]:
def calculate_hash(matrix, shingles):
    for i in range(len(matrix)):
        if matrix[i] in shingles:
            return i

In [15]:
hashes = build_new_hashes()

In [16]:
signatures = []

In [17]:
for i in tqdm(range(docs_num)):
    signature = []
    shingles_set = shingle_sets[i]
    
    for m in hashes:
        cur_hash = calculate_hash(m, shingles_set)
        signature.append(cur_hash)
        
    signatures.append(signature)

100%|████████████████████████████████████| 20016/20016 [26:34<00:00, 12.58it/s]


In [18]:
for i in tqdm(range(docs_num)):
    signature1 = signatures[i]
    for j in range(i + 1, docs_num):
        signature2 = signatures[j]
            
        count = 0
        for k in range(0, num_hashes):
            count += signature1[k] == signature2[k]

        sim_matrix[i][j] = (count / num_hashes)
        sim_matrix[j][i] = (count / num_hashes)

100%|████████████████████████████████████| 20016/20016 [23:46<00:00, 14.03it/s]


In [27]:
signatures[1]

[59509,
 13047,
 654,
 6926,
 37214,
 39352,
 13891,
 30094,
 24278,
 31159,
 22092,
 6818,
 14694,
 42717,
 5328]

In [28]:
signatures[2]

[9846,
 18711,
 42654,
 29611,
 20673,
 4693,
 9932,
 446,
 2984,
 34328,
 2309,
 24075,
 1372,
 14236,
 22056]

In [21]:
threshold = 0.6

In [22]:
i = 1
for j in range(i + 1, docs_num):
    estJ = sim_matrix[i][j]
    
    if estJ > threshold:
        print(j)
        print(news[j]['body'])
        print()